# Residual Convolutional-LSTM


# 準備資料

In [ ]:
!ls DataSet/

In [1]:
import talib as ta
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
from urllib import request
import requests
from time import sleep
import chardet
import sys

def get_file_list(root, ftype = ".csv"):
    import os
    import sys
    FileList = []
    filename = []
    for dirPath, dirNames, fileNames in os.walk(root):
        for f in fileNames:
            if f.find(ftype) > -1:
                FileList.append(os.path.join(dirPath, f))
                filename.append(f.replace(ftype, ""))
    if len(filename) > 0:
        a = zip(FileList, filename)
        a = sorted(a, key = lambda t : t[1])
        FileList, filename = zip(*a)
    return FileList, filename

def crawl3big(date_list):
    if len(date_list) > 0:
        with requests.Session() as s:
            '''s.proxies = {
              'http': 'http://172.104.89.85:8888',
              'https': 'http://161.202.120.89:3128',
            }'''
            s.headers.update({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'})

            for d in date_list:
                sys.stdout.write("\rNow is fetching the data of "+ str(d))
                resp = s.get("http://www.twse.com.tw/fund/T86?response=csv&date="+str(d)+"&selectType=ALL")
                lines = []
                for l in resp.content.decode('big5hkscs').split("\n")[1:]:
                    lines.append(l)

                with open("../TBrain/DataSet/"+str(d)+".csv", "w", encoding='big5hkscs') as f:
                    f.writelines(lines)
                sleep(5)

In [2]:
def squash(x):
    lengths2 = np.sum(np.power(x, 2))
    lengths = np.sqrt(lengths2)
    x = x * np.divide(lengths, (1 + lengths2))
    return x

## 主要資料集處理 - ETF18

In [3]:
etf18 = pd.read_csv("./DataSet/tetfp.csv", encoding="big5hkscs", low_memory=False)
etf18 = etf18.rename(columns={'中文簡稱':'證券名稱'})
etf18 = etf18.dropna(axis=1, how='all')
etf18 = etf18.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
etf18 = etf18.drop_duplicates()
etf18 = etf18.reset_index(drop=True)
code_uq = list(etf18["代碼"].unique())

In [4]:
etf18

,代碼,日期,證券名稱,開盤價(元),最高價(元),最低價(元),收盤價(元),成交張數(張)
0,50,20130102,元大台灣50,54.00,54.65,53.90,54.40,"16,487"
1,50,20130103,元大台灣50,54.90,55.05,54.65,54.85,"29,020"
2,50,20130104,元大台灣50,54.85,54.85,54.40,54.50,"9,837"
3,50,20130107,元大台灣50,54.55,54.55,53.90,54.25,"8,910"
4,50,20130108,元大台灣50,54.00,54.20,53.65,53.90,"12,507"
5,50,20130109,元大台灣50,53.75,54.30,53.75,54.10,"7,529"
6,50,20130110,元大台灣50,54.30,54.65,54.15,54.50,"13,953"
7,50,20130111,元大台灣50,54.70,54.80,54.35,54.45,"11,837"
8,50,20130114,元大台灣50,54.00,54.50,53.80,54.50,"7,282"
9,50,20130115,元大台灣50,54.20,54.45,53.90,54.00,"6,609"


In [5]:
code = etf18.pop("代碼").values
op = etf18.pop("開盤價(元)").values
hi = etf18.pop("最高價(元)").values
lo = etf18.pop("最低價(元)").values
cl = etf18.pop("收盤價(元)").values
vo = etf18.pop("成交張數(張)").values
# names = list(np.unique(etf18["證券名稱"].values))

In [6]:
for i in range(vo.shape[0]):
    vo[i] = float(str(vo[i]).replace(",", ""))
    cl[i] = float(str(cl[i]).replace(",", ""))
    hi[i] = float(str(hi[i]).replace(",", ""))
    lo[i] = float(str(lo[i]).replace(",", ""))
    op[i] = float(str(op[i]).replace(",", ""))

In [7]:
closeDF = pd.DataFrame(np.array(cl).T, columns=["close"])

In [8]:
op_pkg = []
hi_pkg = []
lo_pkg = []
cl_pkg = []
vo_pkg = []
date_list = list(np.unique(etf18["日期"].values))

In [9]:
idx = 0
for i in range(1, len(code)):
    if code[idx] != code[i]:
        op_pkg.append(list(op[idx:i]))
        hi_pkg.append(list(hi[idx:i]))
        lo_pkg.append(list(lo[idx:i]))
        cl_pkg.append(list(cl[idx:i]))
        vo_pkg.append(list(vo[idx:i]))
        idx = i

        
op_pkg.append(list(op[idx:len(code)]))
hi_pkg.append(list(hi[idx:len(code)]))
lo_pkg.append(list(lo[idx:len(code)]))
cl_pkg.append(list(cl[idx:len(code)]))
vo_pkg.append(list(vo[idx:len(code)]))

In [10]:
cl_Box = [[],[],[],[],[]]
for idx, c in enumerate(cl_pkg):
    for i in range(0, 5):
        cl_Box[i] += (list(c[i+1:]) + [np.nan for x in range(min(i+1, len(c)))])

clBoxDF = pd.DataFrame(np.array(cl_Box).T, columns=['cl1', 'cl2', 'cl3', 'cl4', 'cl5'])

In [11]:
closeDF = pd.concat([closeDF, clBoxDF], axis=1)

In [12]:
closeDF

,close,cl1,cl2,cl3,cl4,cl5
0,54.40,54.85,54.50,54.25,53.90,54.10
1,54.85,54.50,54.25,53.90,54.10,54.50
2,54.50,54.25,53.90,54.10,54.50,54.45
3,54.25,53.90,54.10,54.50,54.45,54.50
4,53.90,54.10,54.50,54.45,54.50,54.00
5,54.10,54.50,54.45,54.50,54.00,53.70
6,54.50,54.45,54.50,54.00,53.70,53.30
7,54.45,54.50,54.00,53.70,53.30,53.95
8,54.50,54.00,53.70,53.30,53.95,53.90
9,54.00,53.70,53.30,53.95,53.90,54.05


In [13]:
code_uq

[50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 6201,
 6203,
 6204,
 6208,
 690,
 692,
 701,
 713]

## 次要資料集處理 - 三大法人

In [ ]:
# 爬個資料
lst = []
flist, ftag = get_file_list("../TBrain/DataSet/")
for d in date_list:
    if str(d) not in ftag:
        lst.append(d)        
if len(lst) >0:
    crawl3big(lst)

http://www.twse.com.tw/fund/T86?response=csv&date=20180514&selectType=0099P
http://www.twse.com.tw/fund/T86?response=csv&date=20180514&selectType=ALL

In [ ]:
three = pd.read_csv("../TBrain/DataSet/"+str(date_list[0])+".csv", encoding='big5hkscs')
three.insert(0,'日期',[date_list[0] for i in three.index])
try:
    discrd = three.pop("證券代號")
except:
    print("Error of 證券代號 on "+str(date_list[0]))

three = three.dropna(axis=1, how='all')
three = three.dropna(axis=0,how='any')

for d in date_list[1:]:
    try:
        a = pd.read_csv("./DataSet/"+str(d)+".csv", encoding='big5hkscs')
    except:
        print("Error of OPEN-FILE on "+str(d))
        break
        
    a.insert(0,'日期',[d for i in a.index])
    try:
        discrd = a.pop("證券代號")
    except:
        print("Error of 證券代號 on "+str(d))
        break
        
    a = a.dropna(axis=1, how='all')
    a = a.dropna(axis=0,how='any')
    pd.concat([three, a], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
    three = three.append(a, ignore_index=True)
    
three = three.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
three = three.fillna(0)

In [ ]:
three

In [ ]:
res = pd.merge(etf18, three, on=['日期', '證券名稱'], how='left')
res = res.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
res = res.fillna(0)

In [ ]:
res

In [ ]:
newVal = []
newVal.append(res.pop("日期"))
newVal.append(res.pop("證券名稱"))

newDF = pd.DataFrame(np.array(newVal).T)

In [ ]:
threeVal = res.values
newVal = []
for x in threeVal:
    x2 = []
    for e in x:
        x2.append(float((str(e)).replace(",", "")) / 1000)
    newVal.append(squash(np.array(x2)))
    
new3DF = pd.DataFrame(newVal)

In [ ]:
new3DF

# Pre-proccessing

## KDJ 指標

### KDJ指標的參數設置

KDJ指標參數設置的核心原則：設置參數既需要因人而異，也要因行情而異。
在一般的分析軟體中，KDJ指標的默認參數是（9，3，3）。從實戰的角度來看，由這一參數設置而成的日K線KDJ指標存在著波動頻繁，過於靈敏，無效信號較多的缺陷，也正因為如此，KDJ指標往往被投資者所忽略，認為這一指標並沒有太大的使用價值。但事實上，如果把KDJ指標的參數進行修改，可以發現這一指標對判研價格走勢仍然具有比較好的功效。


#### 1.以（6，3，3）為參數而設置的KDJ指標

對價格波動的敏感性得到加強，它變動的頻率非常高，適合於短線投資者尋找買賣點。一般來說，KDJ三條線在超買超賣區的每一次交叉都可能成為重要的操作時點。
#### 2.以（18，3，3）為參數設置而成的KDJ指標
具有信號穩定而且靈敏度不低的優點，在大多數情況下都比較適用。按照這一參數設定的KDJ指標，有一條非常重要的操作原則就是：在指標處於20超賣區以下，並且指標出現底背離時，應該買進；而在80超賣區以上，指標出現頂背離時應該賣出。
#### 3.以（24，3，3）為參數而設定的KDJ指標
在更大程度上排除了價格波動所產生的虛假信號，用它來尋找價格的中線買點或者是中線賣點是一個比較好的選擇。
#### 參數設置後需要注意的要點：
##### 1.以參數（6，3，3）設置而成的KDJ指標
由於在價格運行中會出現多次交叉，容易產生信號失真，因此需要投資者有足夠的實戰經驗，初學者一般不要輕易嘗試。

##### 2.以參數（18，3，3）設置而成的KDJ指標
在使用中必須遵循這樣的操作原則：
(1)指標的交叉必須是出現在超賣區或者超買區時才是有效信號
(2)在底部發出交叉時，出現兩次交叉應視為良好的買進時機
(3)在高位出現交叉時，出現兩次交叉應視為良好的賣出時機




In [ ]:
def KDJ(high, low, close, rsv = 9, k = 3, d = 3):
    K, D = ta.STOCH(np.array(high), np.array(low), np.array(close), fastk_period=rsv, slowk_period=k, slowk_matype=0, slowd_period=d, slowd_matype=0)
    return K, D, (3 * D - 2 * K)

In [ ]:
paras = [(9,3,3), (6,3,3), (18,3,3), (24, 3, 3), (5,21,11)]
kdj = pd.DataFrame()
for hi, lo, cl in zip(hi_pkg, lo_pkg, cl_pkg):
    box = []
    for p in paras:
        k, d, j = KDJ(hi, lo, cl, p[0], p[1], p[2])
        box.append(k)
        box.append(d)
        box.append(j)
    kdj = kdj.append(pd.DataFrame(data=np.array(box).T), ignore_index=True)

In [ ]:
kdj

In [ ]:
def PriceLoc(close, high, low, period = [6, 11, 22, 43, 65, 130]):
    close = np.array(close)
    high = np.array(high)
    low = np.array(low)
    WCP = ta.WCLPRICE(high, low, close)
    
    sma_cl = []
    for p in period:
        sma_cl.append(ta.SMA(close, timeperiod=p))
    sma_wcl = []
    for p in period:
        sma_wcl.append(ta.SMA(WCP, timeperiod=p))
    
    period.reverse()
    period_r = np.array([period])
    
    wsma_cl = np.sum(period_r.T * sma_cl, axis=0) / sum(period)
    wsma_wcl = np.sum(period_r.T * sma_wcl, axis=0) / sum(period)
    
    space = []
    space.append((WCP - wsma_cl) / wsma_cl)
    space.append((WCP - wsma_wcl) / wsma_wcl)
    for i in range(len(sma_cl)):
        space.append((WCP - sma_cl[i]) / sma_cl[i])
    for i in range(len(sma_wcl)):
        space.append((WCP - sma_wcl[i]) / sma_wcl[i])
        
    return space

In [ ]:
def VolLoc(volume, period = [6, 11, 22, 43, 65, 130]):
    volume = np.array(volume)
    
    smv = []
    for p in period:
        smv.append(ta.SMA(volume, timeperiod=p))
    
    period.reverse()
    period_r = np.array([period])
    
    wsmv = np.sum(period_r.T * smv, axis=0) / sum(period)
    
    space = []
    space.append((volume - wsmv) / wsmv)
    for i in range(len(smv)):
        space.append((volume - smv[i]) / smv[i])
    
    return space

In [ ]:
def StockLoc(close, high, low, volume, period = [6, 11, 22, 43, 65]):
    return PriceLoc(close, high, low, period) + VolLoc(volume, period)

In [ ]:
db = []
for i in range(len(cl_pkg)):
    db.append(StockLoc(cl_pkg[i], hi_pkg[i], lo_pkg[i], vo_pkg[i]))

In [ ]:
loc = pd.DataFrame()
for d in db:
    loc = loc.append(pd.DataFrame(data=np.array(d).T), ignore_index=True)

In [ ]:
loc

In [ ]:
def ClosePercenting(close, period = [1, 2, 3, 4, 5]):
    period.sort()
    cp = []
    for p in period:
        cp.append([])
        
    for i in range(len(close) - period[-1]):
        for j, p in zip(range(len(period)), period):
            cp[j].append((close[i + p] - close[i]) / close[i] * 100)
            
    for i in range(period[-1]):
        for j in range(len(period)):
            cp[j].append(np.nan)
    
    data_dict = {}
    for j in range(len(period)):
        data_dict.update({"change%_D"+str(period[j]) : cp[j]})
    
    out = pd.DataFrame(data=data_dict)
    return out

In [ ]:
change = ClosePercenting(cl_pkg[0])
for cl in cl_pkg[1:]:
    change = change.append(ClosePercenting(cl), ignore_index=True)

In [ ]:
change

### Pattern Recognition Functions

In [ ]:
func_list = [ta.CDL2CROWS, ta.CDL3BLACKCROWS, ta.CDL3INSIDE, ta.CDL3LINESTRIKE, ta.CDL3OUTSIDE, ta.CDL3WHITESOLDIERS, ta.CDLABANDONEDBABY, ta.CDLADVANCEBLOCK, ta.CDLBELTHOLD, ta.CDLBREAKAWAY, ta.CDLCLOSINGMARUBOZU, ta.CDLCONCEALBABYSWALL, ta.CDLCOUNTERATTACK, ta.CDLDARKCLOUDCOVER, ta.CDLDOJI, ta.CDLDOJISTAR, ta.CDLDRAGONFLYDOJI, ta.CDLENGULFING, ta.CDLEVENINGDOJISTAR, ta.CDLEVENINGSTAR, ta.CDLGAPSIDESIDEWHITE, ta.CDLGRAVESTONEDOJI, ta.CDLHANGINGMAN, ta.CDLHARAMI, ta.CDLHARAMICROSS, ta.CDLHIGHWAVE, ta.CDLHIKKAKE, ta.CDLHIKKAKEMOD, ta.CDLHOMINGPIGEON, ta.CDLIDENTICAL3CROWS, ta.CDLINNECK, ta.CDLINVERTEDHAMMER, ta.CDLKICKING, ta.CDLKICKINGBYLENGTH, ta.CDLLADDERBOTTOM, ta.CDLLONGLEGGEDDOJI, ta.CDLLONGLINE, ta.CDLMARUBOZU, ta.CDLMATCHINGLOW, ta.CDLMATHOLD, ta.CDLMORNINGDOJISTAR, ta.CDLMORNINGSTAR, ta.CDLONNECK, ta.CDLPIERCING, ta.CDLRICKSHAWMAN, ta.CDLRISEFALL3METHODS, ta.CDLSEPARATINGLINES, ta.CDLSHOOTINGSTAR, ta.CDLSHORTLINE, ta.CDLSPINNINGTOP, ta.CDLSTALLEDPATTERN, ta.CDLSTICKSANDWICH, ta.CDLTAKURI, ta.CDLTASUKIGAP, ta.CDLTHRUSTING, ta.CDLTRISTAR, ta.CDLUNIQUE3RIVER, ta.CDLUPSIDEGAP2CROWS, ta.CDLXSIDEGAP3METHODS]

In [ ]:
ptnDF = pd.DataFrame()
for i in range(len(code_uq)):
    a = []
    for f in func_list:
        a.append(f(np.array(op_pkg[i]), np.array(hi_pkg[i]), np.array(lo_pkg[i]), np.array(cl_pkg[i])))

    ptnDF = ptnDF.append(pd.DataFrame(np.array(a).T), ignore_index=True) 
    
ptnVal = ptnDF.values
ptnValSQ = []
for x in ptnVal:
    ptnValSQ.append(squash(x))

ptnDFSQ = pd.DataFrame(ptnValSQ)

In [ ]:
ptnDFSQ

In [ ]:
finalETF18 = pd.concat([etf18, change, closeDF, loc, kdj, new3DF, ptnDFSQ], axis=1)

In [ ]:
finalETF18

In [ ]:
finalETF18.to_csv("../TBrain/DataSet/final18_"+str(date_list[-1])+".csv", index=False)

# Prepare for Training

In [14]:
finalETF18 = pd.read_csv("../TBrain/DataSet/final18_"+str(date_list[-1])+".csv")
finalETF18 = finalETF18.dropna() # 去除空資訊
finalETF18 = finalETF18.drop_duplicates()
discard = finalETF18.pop("日期")# 移除日期
tags = finalETF18.pop("證券名稱").values # 取得標籤
dataVal = finalETF18.values

In [15]:
dataValSplit = []

idx = 0
for i in range(1, len(tags)):
    if tags[idx] != tags[i]:
        dataValSplit.append(list(dataVal[idx:i]))
        idx = i

dataValSplit.append(list(dataVal[idx:len(dataVal) - 1]))

In [16]:
frames = []
for d in dataValSplit:
    frames.append(pd.DataFrame(d))

In [17]:
frames[0]

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
0,0.185529,0.000000,-0.092764,2.875696,2.782931,53.90,54.00,53.90,53.85,55.45,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,-0.185185,-0.277778,2.685185,2.592593,2.037037,54.00,53.90,53.85,55.45,55.40,...,0.000000,0.0,0.0,0.000000,0.000000,-0.577331,0.0,0.0,0.0,0.0
2,-0.092764,2.875696,2.782931,2.226345,3.061224,53.90,53.85,55.45,55.40,55.10,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,2.971216,2.878366,2.321263,3.156917,3.342618,53.85,55.45,55.40,55.10,55.55,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,-0.090171,-0.631199,0.180343,0.360685,0.721371,55.45,55.40,55.10,55.55,55.65,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
5,-0.541516,0.270758,0.451264,0.812274,0.722022,55.40,55.10,55.55,55.65,55.85,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
6,0.816697,0.998185,1.361162,1.270417,2.359347,55.10,55.55,55.65,55.85,55.80,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
7,0.180018,0.540054,0.450045,1.530153,1.800180,55.55,55.65,55.85,55.80,56.40,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8,0.359389,0.269542,1.347709,1.617251,1.796945,55.65,55.85,55.80,56.40,56.55,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
9,-0.089526,0.984781,1.253357,1.432408,1.880036,55.85,55.80,56.40,56.55,56.65,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [18]:
Y_box = []
for f in frames:
    tmp = []
    for i in range(0, 11):
        tmp.append(f.pop(i).values)
    Y_box.append(tmp)

In [19]:
Y_frames = []
for y in Y_box:
    Y_frames.append(pd.DataFrame(np.array(y).T, columns=['d1', 'd2', 'd3', 'd4', 'd5', 'close', 'cl1', 'cl2', 'cl3', 'cl4', 'cl5']))

In [20]:
Y_frames[0]

,d1,d2,d3,d4,d5,close,cl1,cl2,cl3,cl4,cl5
0,0.185529,0.000000,-0.092764,2.875696,2.782931,53.90,54.00,53.90,53.85,55.45,55.40
1,-0.185185,-0.277778,2.685185,2.592593,2.037037,54.00,53.90,53.85,55.45,55.40,55.10
2,-0.092764,2.875696,2.782931,2.226345,3.061224,53.90,53.85,55.45,55.40,55.10,55.55
3,2.971216,2.878366,2.321263,3.156917,3.342618,53.85,55.45,55.40,55.10,55.55,55.65
4,-0.090171,-0.631199,0.180343,0.360685,0.721371,55.45,55.40,55.10,55.55,55.65,55.85
5,-0.541516,0.270758,0.451264,0.812274,0.722022,55.40,55.10,55.55,55.65,55.85,55.80
6,0.816697,0.998185,1.361162,1.270417,2.359347,55.10,55.55,55.65,55.85,55.80,56.40
7,0.180018,0.540054,0.450045,1.530153,1.800180,55.55,55.65,55.85,55.80,56.40,56.55
8,0.359389,0.269542,1.347709,1.617251,1.796945,55.65,55.85,55.80,56.40,56.55,56.65
9,-0.089526,0.984781,1.253357,1.432408,1.880036,55.85,55.80,56.40,56.55,56.65,56.90


In [21]:
X_box = []
for f in frames:
    X_box.append(f.values)

In [22]:
def train_test_split(data_box, target_box, train_rate=.9, test_rate=.1):
    x_train = []
    x_test = []
    y_train = []
    y_test = []
    if (train_rate + test_rate != 1):
        test_rate = 1 - train_rate
    
    for i in range(len(data_box)):
        tot = len(data_box[i]) # 總長度
        tra = int(np.ceil(tot * train_rate) - 1)
        x_train.append(list(data_box[i][:tra]))
        x_test.append(list(data_box[i][tra:]))
        
        tmp_ytr = []
        tmp_yte = []
        for j in range(len(target_box[i])):
            tmp_ytr.append(list(target_box[i][j][:tra]))
            tmp_yte.append(list(target_box[i][j][tra:]))
        y_train.append(tmp_ytr)
        y_test.append(tmp_yte)
        
    return x_train, y_train, x_test, y_test

In [23]:
x_train, y_train, x_test, y_test = train_test_split(X_box, Y_box, 0.9)

In [24]:
x_50tr, y_50tr, x_50te, y_50te = train_test_split(X_box, Y_box, 0.5)
x_ftr, y_ftr, x_fte, y_fte = train_test_split(x_50te, y_50te, 0.8)

In [25]:
for i_fte, fte in enumerate(x_fte):
    print(i_fte, np.array(fte).shape)

0 (128, 114)
1 (128, 114)
2 (128, 114)
3 (128, 114)
4 (128, 114)
5 (128, 114)
6 (128, 114)
7 (128, 114)
8 (128, 114)
9 (128, 114)
10 (128, 114)
11 (127, 114)
12 (127, 114)
13 (128, 114)
14 (24, 114)
15 (21, 114)
16 (14, 114)
17 (11, 114)


In [26]:
y_fte[-1]

[[0.0,
  0.51612903225806495,
  -0.12836970474967632,
  -1.1889460154241676,
  0.19512195121950804,
  0.48685491723467111,
  0.77519379844960734,
  0.48076923076923772,
  0.31897926634768059,
  -0.19077901430842198,
  -0.28671551449506172],
 [0.51612903225806495,
  0.3870967741935516,
  -1.3157894736842108,
  -0.99614395886890184,
  0.68292682926829551,
  1.2658227848101284,
  1.2596899224806222,
  0.80128205128205143,
  0.12759170653907226,
  -0.47694753577106058,
  -0.19114367633004864],
 [0.3870967741935516,
  -0.80645161290322576,
  -1.1232349165596964,
  -0.51413881748072032,
  1.4634146341463392,
  1.7526777020447994,
  1.5826873385012867,
  0.60897435897436292,
  -0.15948963317384596,
  -0.38155802861685528,
  -0.4460019114367651],
 [-0.80645161290322576,
  -0.61290322580645573,
  -0.64184852374839307,
  0.25706940874035439,
  1.9512195121951263,
  2.0772476468679022,
  1.388888888888888,
  0.32051282051282504,
  -0.063795853269547453,
  -0.63593004769475137,
  -0.38228735266008

In [27]:
y_train[-1][-1][-1]

30.75

In [28]:
import torch
from torch.autograd import Variable as var
from torch import nn, cuda
from torch.utils.data import Dataset, DataLoader, sampler

In [29]:
class DSet(Dataset):
    def __init__(self, x, y):
        super(DSet, self).__init__()
        self.base_x = x
        self.base_y = y
    def __getitem__(self, index):
        while index < 0:
            index += self.__len__()
        while index >= self.__len__():
            index -= self.__len__()

        for i in range(len(self.base_x)):
            if index > len(self.base_x[i]) - 1 - 8:
                index -= (len(self.base_x[i]) - 1 - 8)
            else:
                # y will output ['d1', 'd2', 'd3', 'd4', 'd5', 'close', 'cl1', 'cl2', 'cl3', 'cl4', 'cl5']
                y = np.array([self.base_y[i][x][index + 8] for x in range(len(self.base_y[i]))])
                return {"X":(torch.FloatTensor([self.base_x[i][index : index + 9]])), "Y":y, "close":(torch.FloatTensor([self.base_y[i][5][index + 8]]))}

    def __len__(self):
        tot = 0
        for x in self.base_x:
            if (len(x) - 9) >= 0:
                tot += (len(x) - 9)
            else:
                tot += 0
        return tot + 1

In [30]:
class DSet2(Dataset): # for fine tune
    def __init__(self, x, y):
        super(DSet2, self).__init__()
        self.base_x = x
        self.base_y = y
    def __getitem__(self, index):
        while index < 0:
            index += self.__len__()
        while index >= self.__len__():
            index -= self.__len__()


        # y will output ['d1', 'd2', 'd3', 'd4', 'd5', 'close', 'cl1', 'cl2', 'cl3', 'cl4', 'cl5']
        y = np.array([self.base_y[x][index + 8] for x in range(len(self.base_y))])
        return {"X":(torch.FloatTensor([self.base_x[index : index + 9]])), "Y":y, "close":(torch.FloatTensor([self.base_y[5][index + 8]]))}


    def __len__(self):
        tot = 0
        tot += (len(self.base_x) - 9)
        if tot < 0:
            tot = 0
        return tot + 1

In [31]:
train = DSet(x_train, y_train)
valid = DSet(x_test, y_test)

finetune_tr = []
for i in range(len(x_ftr)):
    finetune_tr.append(DSet2(x_ftr[i], y_ftr[i]))

finetune_te = []
for i in range(len(x_fte)):
    finetune_te.append(DSet2(x_fte[i], y_fte[i]))

In [32]:
train[1]

{'X': 
 ( 0 ,.,.) = 
   -0.0078  -0.0078  -0.0029  ...    0.0000   0.0000   0.0000
   -0.0058  -0.0058  -0.0021  ...    0.0000   0.0000   0.0000
   -0.0070  -0.0069  -0.0041  ...    0.0000   0.0000   0.0000
             ...               ⋱              ...            
    0.0148   0.0153   0.0118  ...    0.0000   0.0000   0.0000
    0.0129   0.0135   0.0081  ...    0.0000   0.0000   0.0000
    0.0150   0.0154   0.0077  ...    0.0000   0.0000   0.0000
 [torch.FloatTensor of size 1x9x114],
 'Y': array([ -0.08952551,   0.98478066,   1.25335721,   1.43240824,
          1.88003581,  55.85      ,  55.8       ,  56.4       ,
         56.55      ,  56.65      ,  56.9       ]),
 'close': 
  55.8500
 [torch.FloatTensor of size 1]}

In [33]:
print(len(train))
train[16084]

16407


{'X': 
 ( 0 ,.,.) = 
     0.0099    0.0100    0.0019  ...     0.0000    0.0000    0.0000
     0.0077    0.0079    0.0002  ...     0.0000    0.0000    0.0000
     0.0051    0.0052   -0.0016  ...     0.0000    0.0000    0.0000
               ...                ⋱                ...             
     0.0075    0.0075    0.0034  ...     0.0000    0.0000    0.0000
     0.0078    0.0078    0.0039  ...     0.0000    0.0000    0.0000
     0.0068    0.0067    0.0029  ...     0.0000    0.0000    0.0000
 [torch.FloatTensor of size 1x9x114],
 'Y': array([ -0.0452284 ,  -0.09045681,  -0.09045681,   0.13568521,
          0.0452284 ,  22.11      ,  22.1       ,  22.09      ,
         22.09      ,  22.14      ,  22.12      ]),
 'close': 
  22.1100
 [torch.FloatTensor of size 1]}

# Model Building

In [34]:
log_softmax = lambda x : np.round(np.log(np.exp(x) / np.sum(np.exp(x))), 4)
def Build_Y(cp_1D, classes = [x for x in np.arange(-9.5, 10, 1)]):
    out = []
    for x in cp_1D:
        tmp = []
        for c in classes:
            tmp.append(-(abs(x - c)))
        out.append(log_softmax(tmp))
    return out

In [35]:
class simpleCRNN(nn.Module):
    def __init__(self):
        super(simpleCRNN, self).__init__()
        # torch.Size([9, 1, 40])
        # torch.Size([1, 1, 360])
        self.lstm0 = nn.LSTM(
            input_size = 114,
            hidden_size = 20,
            num_layers = 1,
            bidirectional = True
        )
        self.h0 = None
        
        # torch.Size([7, 1, 40])
        # torch.Size([1, 1, 280])
        self.conv1 = nn.Conv2d(1, 1, 3)
        self.lstm1 = nn.LSTM(
            input_size = 112,
            hidden_size = 20,
            num_layers = 1,
            bidirectional = True
        )
        self.h1 = None
        
        # torch.Size([4, 1, 40])
        # torch.Size([1, 1, 160])
        self.conv2 = nn.Conv2d(1, 1, 6)
        self.lstm2 = nn.LSTM(
            input_size = 109,
            hidden_size = 20,
            num_layers = 1,
            bidirectional = True
        )
        self.h2 = None
        
        # torch.Size([1, 1, 40])
        self.conv3 = nn.Conv2d(1, 1, 9)
        self.lstm3 = nn.LSTM(
            input_size = 106,
            hidden_size = 20,
            num_layers = 1,
            bidirectional = True
        )
        self.h3 = None
        
        self.resizer = [nn.Linear(840, 1)for i in range(5)]
        for m in range(len(self.resizer)):
            self.add_module('resizer-'+str(m + 1), self.resizer[m])
    def forward(self, x, val):
        out0, self.h0 = self.lstm0(x.view(9, 1, -1), self.h0)
        out = out0.view(1, -1)
        
        c = self.conv1(x.view(1, 1, 9, -1)).view(7, 1, -1)
        out1, self.h1 = self.lstm1(c, self.h1)
        out = torch.cat((out, out1.view(1, -1)), 1)
        
        c = self.conv2(x.view(1, 1, 9, -1)).view(4, 1, -1)
        out2, self.h2 = self.lstm2(c, self.h2)
        out = torch.cat((out, out2.view(1, -1)), 1)
        
        c = self.conv3(x.view(1, 1, 9, -1)).view(1, 1, -1)
        out3, self.h3 = self.lstm3(c, self.h3)
        out = torch.cat((out, out3.view(1, -1)), 1)
        
        updn = []
        for i_r, r in enumerate(self.resizer):
            updn.append(r(out.view(1, -1)))
        
        ans = []
        for i_ud, ud in enumerate(updn):
            ans.append(torch.mul(val.view(1, -1), ud))
        
        updn[0] = torch.sign(ans[0] - val)
        for i in range(1, len(ans)):
            updn[i] = torch.sign(ans[i] - ans[i - 1])
        
        return updn, ans
        
        

In [36]:
net = nn.DataParallel(simpleCRNN().cuda())
try:
    net.load_state_dict(torch.load("./cr2.pth"))
except:
    try:
        net.load_state_dict(torch.load("./cr2_base.pth"))
    except:
        print("NO pre-trained model.")
    else:
        print("Last State has load!")
else:
    print("Last State has load!")

Last State has load!


In [37]:
net

DataParallel(
  (module): simpleCRNN(
    (lstm0): LSTM(114, 20, bidirectional=True)
    (conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (lstm1): LSTM(112, 20, bidirectional=True)
    (conv2): Conv2d(1, 1, kernel_size=(6, 6), stride=(1, 1))
    (lstm2): LSTM(109, 20, bidirectional=True)
    (conv3): Conv2d(1, 1, kernel_size=(9, 9), stride=(1, 1))
    (lstm3): LSTM(106, 20, bidirectional=True)
    (resizer-1): Linear(in_features=840, out_features=1, bias=True)
    (resizer-2): Linear(in_features=840, out_features=1, bias=True)
    (resizer-3): Linear(in_features=840, out_features=1, bias=True)
    (resizer-4): Linear(in_features=840, out_features=1, bias=True)
    (resizer-5): Linear(in_features=840, out_features=1, bias=True)
  )
)

# Training

In [38]:
Learning_Rate = 1e-3
EPOCH = 0
BATACH_SIZE = 22

In [39]:

train_dl = DataLoader(dataset=train, batch_size=BATACH_SIZE, num_workers=6, sampler=sampler.SequentialSampler(train), pin_memory=True)
#train_dl = DataLoader(dataset=train, batch_size=BATACH_SIZE, num_workers=6, pin_memory=True, shuffle=True)
valid_dl = DataLoader(dataset=valid, batch_size=5, num_workers=6, sampler=sampler.SequentialSampler(valid), pin_memory=True)

finetune_tr_dl = []
finetune_te_dl = []
for i_ftr, ftr in enumerate(finetune_tr):
    finetune_tr_dl.append(DataLoader(dataset=ftr, batch_size=20, num_workers=6, sampler=sampler.SequentialSampler(ftr), pin_memory=True))
for i_fte, fte in enumerate(finetune_te):
    finetune_te_dl.append(DataLoader(dataset=fte, batch_size=5, num_workers=6, sampler=sampler.SequentialSampler(fte), pin_memory=True))


In [40]:
op = torch.optim.Adam(net.parameters(), lr=Learning_Rate)
lf1 = nn.KLDivLoss()
lf2 = nn.MSELoss()
lf3 = nn.SmoothL1Loss()
lf4 = lambda a, b : 1 if torch.equal(a.view(1, -1), b.view(1, -1)) else 0
w = [0.1, 0.15, 0.2, 0.25, 0.3]
_100 = var(torch.FloatTensor([100])).cuda()

In [41]:
epo_score = []
max_va = -1
no_up = 0
for epo in range(EPOCH):
    print("Epoch Now:",epo+1,",Total:",EPOCH)
    scores = []
    for i_batch, sample_batched in enumerate(train_dl):
        for mini_batch in range(sample_batched['X'].size()[0]):
            x = var(sample_batched['X'][mini_batch]).cuda()
            val = var(sample_batched['close'][mini_batch]).cuda()
            updn, ans = net(x, val)
            op.zero_grad()
            
            y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][0 + 6]]])).cuda()
            y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + 0] - sample_batched['Y'][mini_batch][5 + 0 + 0]])))).view(1,1,-1).cuda()
            score = w[0] * (50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3))
            loss = lf3((50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3)).view(1, -1), _100.view(1, -1))
            for i_sm in range(1, 5):
                y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][i_sm + 6]]])).cuda()
                y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + i_sm] - sample_batched['Y'][mini_batch][5 + 0 + i_sm]])))).view(1,1,-1).cuda()
                score += w[i_sm] * (50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3))
                loss += lf3((50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3)).view(1, -1), _100.view(1, -1))
            
            loss.backward()
            op.step()
            scores.append(score.cpu().data.numpy()[0])
            cuda.empty_cache()
            
        if (i_batch + 1) % 500 == 0:
            print("500 Batches Trained!, Avg score = {0}".format(np.average(scores[-100:])))
            #torch.save(net.state_dict(), "./cr2_base_E{0}_B{1}.pth".format(epo, i_batch + 1))
    
    cuda.empty_cache()
    print("Train score", np.average(scores))
    scores = []
    for i_batch, sample_batched in enumerate(valid_dl):
        for mini_batch in range(sample_batched['X'].size()[0]):
            x = var(sample_batched['X'][mini_batch]).cuda()
            val = var(sample_batched['close'][mini_batch]).cuda()
            updn, ans = net(x, val)
            op.zero_grad()
            
            y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][0 + 6]]])).cuda()
            y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + 0] - sample_batched['Y'][mini_batch][5 + 0 + 0]])))).view(1,1,-1).cuda()
            score = w[0] * (50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3))
            for i_sm in range(1, 5):
                y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][i_sm + 6]]])).cuda()
                y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + i_sm] - sample_batched['Y'][mini_batch][5 + 0 + i_sm]])))).view(1,1,-1).cuda()
                score += w[i_sm] * (50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3))
            scores.append(score.cpu().data.numpy()[0])
            cuda.empty_cache()
    epo_score.append(np.average(scores))
    print("Valid score", epo_score[-1])
    
    if epo_score[-1] > max_va:
        no_up = 0
        max_va = epo_score[-1]
    else:
        no_up += 1
    
    torch.save(net.state_dict(), "./cr2.pth")
    if no_up >= 10:
        print("\n End at the {0} epoch".format(epo+1))
        break


# Model Validation

In [42]:
Result = []
for i_file in range(len(finetune_tr_dl)):
    print("\nFile#", i_file + 1, "Finetuning")
    try:
        net.load_state_dict(torch.load("./cr2_final_{}_{}.pth".format(date_list[-6],code_uq[i_file])))
        print("Load", "./cr2_final_{}_{}.pth".format(date_list[-6],code_uq[i_file]))
        trEPOCH = 100
        teEPOCH = 200
    except Exception as e:
        print(str(e))
        net.load_state_dict(torch.load("./cr2.pth"))
        print("Load Base")
        trEPOCH = 300
        teEPOCH = 400
    
    op = torch.optim.Adam(net.parameters(), lr=Learning_Rate)
    epo_score = []
    max_va = -1e9
    no_up = 0
    for epo in range(trEPOCH):
        print("epoch#", epo + 1)
        scores = []
        for i_batch, sample_batched in enumerate(finetune_tr_dl[i_file]):
            for mini_batch in range(sample_batched['X'].size()[0]):
                x = var(sample_batched['X'][mini_batch]).cuda()
                val = var(sample_batched['close'][mini_batch]).cuda()
                updn, ans = net(x, val)
                op.zero_grad()

                y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][0 + 6]]])).cuda()
                y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + 0] - sample_batched['Y'][mini_batch][5 + 0 + 0]])))).view(1,1,-1).cuda()
                score = w[0] * (50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3))
                loss = lf3((50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3)).view(1, -1), _100.view(1, -1))
                for i_sm in range(1, 5):
                    y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][i_sm + 6]]])).cuda()
                    y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + i_sm] - sample_batched['Y'][mini_batch][5 + 0 + i_sm]])))).view(1,1,-1).cuda()
                    score += w[i_sm] * (50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3))
                    loss += lf3((50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3)).view(1, -1), _100.view(1, -1))

                loss.backward(retain_graph = True)
                op.step()
                scores.append(score.cpu().data.numpy()[0])
                cuda.empty_cache()
        epo_score.append(np.average(scores))
        print("score", epo_score[-1])
        valid_tmp = []
        for i_batch, sample_batched in enumerate(finetune_te_dl[i_file]):
            for mini_batch in range(sample_batched['X'].size()[0]):
                x = var(sample_batched['X'][mini_batch]).cuda()
                val = var(sample_batched['close'][mini_batch]).cuda()
                updn, ans = net(x, val)

                y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][0 + 6]]])).cuda()
                y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + 0] - sample_batched['Y'][mini_batch][5 + 0 + 0]])))).view(1,1,-1).cuda()
                score = w[0] * (50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3))
                for i_sm in range(1, 5):
                    y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][i_sm + 6]]])).cuda()
                    y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + i_sm] - sample_batched['Y'][mini_batch][5 + 0 + i_sm]])))).view(1,1,-1).cuda()
                    score += w[i_sm] * (50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3))

                valid_tmp.append(score.cpu().data.numpy()[0])
                
        print("Valid:", np.average(valid_tmp))
        if np.average(valid_tmp) > max_va:
            no_up = 0
            max_va = np.average(valid_tmp)
        else:
            no_up += 1
            
        if no_up >= 10:
            print("Fintune has done! Average Score is {0} / 1".format(np.average(epo_score) / 100))
            break
    pass
    print("Valid Model...")
    for p in net.parameters():
        p.requires_grad = False
    valid_tmp = []
    for i_batch, sample_batched in enumerate(finetune_te_dl[i_file]):
        for mini_batch in range(sample_batched['X'].size()[0]):
            x = var(sample_batched['X'][mini_batch]).cuda()
            val = var(sample_batched['close'][mini_batch]).cuda()
            updn, ans = net(x, val)

            y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][0 + 6]]])).cuda()
            y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + 0] - sample_batched['Y'][mini_batch][5 + 0 + 0]])))).view(1,1,-1).cuda()
            score = w[0] * (50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3))
            for i_sm in range(1, 5):
                y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][i_sm + 6]]])).cuda()
                y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + i_sm] - sample_batched['Y'][mini_batch][5 + 0 + i_sm]])))).view(1,1,-1).cuda()
                score += w[i_sm] * (50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3))

            valid_tmp.append(score.cpu().data.numpy()[0])
        print("Batch#{0}, Score = {1} / 1".format(i_batch, np.round(np.average(valid_tmp[-sample_batched['X'].size()[0] : ]), 6)))
    pass
    for p in net.parameters():
        p.requires_grad = True
    print("Fintune again for predict next week...")
    epo_score = []
    no_up = 0
    max_va = -1e9
    for epo in range(teEPOCH):
        print("epoch#", epo + 1)
        scores = []
        for i_batch, sample_batched in enumerate(finetune_te_dl[i_file]):
            for mini_batch in range(sample_batched['X'].size()[0]):
                x = var(sample_batched['X'][mini_batch]).cuda()
                val = var(sample_batched['close'][mini_batch]).cuda()
                updn, ans = net(x, val)
                op.zero_grad()

                y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][0 + 6]]])).cuda()
                y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + 0] - sample_batched['Y'][mini_batch][5 + 0 + 0]])))).view(1,1,-1).cuda()
                score = w[0] * (50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3))
                loss = lf3((50 * ((y2 - torch.abs(ans[0] - y2)) / y2) + 50 * lf4(updn[0], y3)).view(1, -1), _100.view(1, -1))
                for i_sm in range(1, 5):
                    y2 = var(torch.FloatTensor([[sample_batched['Y'][mini_batch][i_sm + 6]]])).cuda()
                    y3 = var(torch.FloatTensor(np.sign(np.array([sample_batched['Y'][mini_batch][5 + 1 + i_sm] - sample_batched['Y'][mini_batch][5 + 0 + i_sm]])))).view(1,1,-1).cuda()
                    score += w[i_sm] * (50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3))
                    loss += lf3((50 * ((y2 - torch.abs(ans[i_sm] - y2)) / y2) + 50 * lf4(updn[i_sm], y3)).view(1, -1), _100.view(1, -1))

                loss.backward(retain_graph = True)
                op.step()
                scores.append(score.cpu().data.numpy()[0])
                cuda.empty_cache()
        epo_score.append(np.average(scores))
        print("score", epo_score[-1])
        if epo_score[-1] > max_va:
            no_up = 0
            max_va = epo_score[-1]
        else:
            no_up += 1
            
        if no_up >= 3:
            print("Fintune has done! Average Score is {0} / 1".format(np.average(epo_score) / 100))
            break
    pass
    print("Prepare to predict")
    raw = pd.read_csv("../TBrain/DataSet/final18_"+str(date_list[-1])+".csv")
    val = raw.values
    thisweek = []
    idx = 0
    for i in range(1, len(val)):
        if val[i][1] != val[idx][1]:
            thisweek.append(val[i - 9:i])
            idx = i
    thisweek.append(val[len(val) - 9:])
    thisweek2 = []
    for i in range(len(thisweek)):
        tmp = []
        for ii in range(len(thisweek[i])):
            tmp.append(thisweek[i][ii][13:])
        thisweek2.append(tmp)

    x = var(torch.FloatTensor(thisweek2[i_file])).view(1, 9, -1).cuda()
    val = var(torch.FloatTensor([thisweek[i_file][-1][7]])).cuda()
    updn, ans = net(x, val)

    res = []
    res.append(code_uq[i_file])
    for ans, ud in zip(ans, updn):
        res.append(ud.view(-1).cpu().data.numpy()[0])
        res.append(ans.view(-1).cpu().data.numpy()[0])
    Result.append(res)
    torch.save(net.state_dict(), "./cr2_final_{0}_{1}.pth".format(date_list[-1], code_uq[i_file]))
    cuda.empty_cache()
    print("Result:", res)

result = pd.DataFrame(Result, columns=["ETFid","Mon_ud","Mon_cprice","Tue_ud","Tue_cprice", "Wed_ud","Wed_cprice","Thu_ud","Thu_cprice","Fri_ud","Fri_cprice"])
result.to_csv("./cr2_result_"+str(date_list[-1])+".csv", index=False)


File# 1 Finetuning
Load ./cr2_final_20180608_50.pth
epoch# 1
score 72.6211
Valid: 74.8559
epoch# 2
score 72.401
Valid: 74.5486
epoch# 3
score 72.4317
Valid: 74.5612
epoch# 4
score 73.4865
Valid: 74.8003
epoch# 5
score 72.382
Valid: 73.0201
epoch# 6
score 72.4474
Valid: 74.9401
epoch# 7
score 72.6669
Valid: 72.8646
epoch# 8
score 72.4299
Valid: 74.8739
epoch# 9
score 72.31
Valid: 74.0224
epoch# 10
score 72.0057
Valid: 72.713
epoch# 11
score 72.4963
Valid: 72.5721
epoch# 12
score 72.2652
Valid: 75.1946
epoch# 13
score 73.2009
Valid: 71.8463
epoch# 14
score 72.9564
Valid: 75.4222
epoch# 15
score 72.5324
Valid: 72.9116
epoch# 16
score 72.7152
Valid: 73.4104
epoch# 17
score 73.0003
Valid: 72.9849
epoch# 18
score 72.5044
Valid: 71.4151
epoch# 19
score 72.5838
Valid: 75.2816
epoch# 20
score 72.6284
Valid: 73.4725
epoch# 21
score 72.6626
Valid: 72.8265
epoch# 22
score 72.9323
Valid: 73.3595
epoch# 23
score 72.6461
Valid: 74.8018
epoch# 24
score 72.722
Valid: 71.6936
Fintune has done! Average 

score 71.8223
Valid: 67.7643
epoch# 4
score 72.0175
Valid: 69.1821
epoch# 5
score 72.242
Valid: 67.5669
epoch# 6
score 72.0732
Valid: 68.4421
epoch# 7
score 72.5709
Valid: 69.2949
epoch# 8
score 71.3922
Valid: 68.6715
epoch# 9
score 71.5622
Valid: 68.6203
epoch# 10
score 72.659
Valid: 68.5553
epoch# 11
score 71.6028
Valid: 69.2932
epoch# 12
score 72.4772
Valid: 68.7521
epoch# 13
score 72.8834
Valid: 68.6446
epoch# 14
score 72.1004
Valid: 69.0507
epoch# 15
score 72.3961
Valid: 69.7402
epoch# 16
score 72.4742
Valid: 68.9906
epoch# 17
score 72.355
Valid: 70.1893
epoch# 18
score 71.6875
Valid: 68.7592
epoch# 19
score 72.5664
Valid: 68.3579
epoch# 20
score 72.6082
Valid: 69.2129
epoch# 21
score 72.557
Valid: 69.5764
epoch# 22
score 71.7692
Valid: 68.4687
epoch# 23
score 72.8631
Valid: 69.4166
epoch# 24
score 72.3227
Valid: 68.8672
epoch# 25
score 72.7686
Valid: 67.6837
epoch# 26
score 71.7049
Valid: 68.7122
epoch# 27
score 73.0041
Valid: 68.0823
Fintune has done! Average Score is 0.72252769

score 70.7197
epoch# 2
score 68.7536
epoch# 3
score 70.5823
epoch# 4
score 71.7324
epoch# 5
score 70.0145
epoch# 6
score 68.0897
epoch# 7
score 70.8826
Fintune has done! Average Score is 0.7011067962646484 / 1
Prepare to predict
Result: [57, 1.0, 50.462372, -1.0, 49.686806, 1.0, 50.907005, 1.0, 52.899609, -1.0, 51.724197]

File# 9 Finetuning
Load ./cr2_final_20180608_58.pth
epoch# 1
score 67.9861
Valid: 69.3283
epoch# 2
score 68.0158
Valid: 70.1664
epoch# 3
score 67.3215
Valid: 69.5238
epoch# 4
score 67.8784
Valid: 69.5252
epoch# 5
score 67.9835
Valid: 69.9972
epoch# 6
score 67.9211
Valid: 70.3111
epoch# 7
score 67.2682
Valid: 70.2927
epoch# 8
score 67.4923
Valid: 69.4902
epoch# 9
score 68.0213
Valid: 70.1576
epoch# 10
score 67.8932
Valid: 69.9544
epoch# 11
score 68.0228
Valid: 69.7531
epoch# 12
score 67.2164
Valid: 70.4423
epoch# 13
score 67.5457
Valid: 69.348
epoch# 14
score 68.0842
Valid: 69.9771
epoch# 15
score 67.7418
Valid: 69.5121
epoch# 16
score 68.3732
Valid: 69.8457
epoch# 17

score 70.0923
epoch# 2
score 68.8239
epoch# 3
score 69.5893
epoch# 4
score 68.284
Fintune has done! Average Score is 0.6919740295410156 / 1
Prepare to predict
Result: [6203, -1.0, 37.936348, 1.0, 38.759789, 1.0, 39.373451, -1.0, 38.466057, 1.0, 39.246765]

File# 13 Finetuning
Load ./cr2_final_20180608_6204.pth
epoch# 1
score 68.0815
Valid: 68.1698
epoch# 2
score 67.7712
Valid: 69.7129
epoch# 3
score 67.6276
Valid: 69.8406
epoch# 4
score 68.0163
Valid: 67.5579
epoch# 5
score 68.7301
Valid: 68.1107
epoch# 6
score 67.7322
Valid: 67.467
epoch# 7
score 67.825
Valid: 67.9883
epoch# 8
score 67.213
Valid: 69.4602
epoch# 9
score 68.2139
Valid: 68.3247
epoch# 10
score 68.0187
Valid: 68.3335
epoch# 11
score 67.8981
Valid: 68.504
epoch# 12
score 67.3393
Valid: 68.8358
epoch# 13
score 67.9537
Valid: 67.1346
Fintune has done! Average Score is 0.6787850952148438 / 1
Valid Model...
Batch#0, Score = 58.51042938232422 / 1
Batch#1, Score = 69.71903991699219 / 1
Batch#2, Score = 64.78629302978516 / 1
Batc

Result: [701, -1.0, 21.078434, 1.0, 21.284033, -1.0, 20.992483, 1.0, 21.172592, 1.0, 21.504948]

File# 18 Finetuning
Load ./cr2_final_20180608_713.pth
epoch# 1
score 70.7543
Valid: 73.6471
epoch# 2
score 69.0072
Valid: 71.2354
epoch# 3
score 70.0739
Valid: 76.3359
epoch# 4
score 68.0327
Valid: 80.3347
epoch# 5
score 70.7557
Valid: 82.9208
epoch# 6
score 70.6324
Valid: 80.5066
epoch# 7
score 68.7747
Valid: 79.4898
epoch# 8
score 69.3065
Valid: 80.5014
epoch# 9
score 69.8704
Valid: 82.8934
epoch# 10
score 69.2997
Valid: 75.3243
epoch# 11
score 67.4421
Valid: 70.6163
epoch# 12
score 67.9355
Valid: 78.0781
epoch# 13
score 67.762
Valid: 68.9538
epoch# 14
score 69.6113
Valid: 73.6465
epoch# 15
score 70.6026
Valid: 70.5682
Fintune has done! Average Score is 0.6932408142089844 / 1
Valid Model...
Batch#0, Score = 70.56820678710938 / 1
Fintune again for predict next week...
epoch# 1
score 67.1105
epoch# 2
score 76.3505
epoch# 3
score 77.1998
epoch# 4
score 73.7549
epoch# 5
score 78.0087
epoch# 6

In [ ]:
!ls |grep cr2_result